# Relational Databases with Julia

This notebook contains some basic functionallity for using relational databases with Julia.
More specifically, we focus on PostgreSQL.

In [1]:
using Pkg
Pkg.activate(".")
using LibPQ
using SQLStrings

  Activating project at `~/Main/EMAp/Julia_Tutorials/DataBases`


In [2]:
using DataFrames
using PalmerPenguins
using Tables

## 1. Setting Up PostgreSQL Locally

Before we can start, you need to install postgres in your computer. I'm assuming you are using Linux
with Ubuntu. Hop into the terminal and run the following commands.
First, run `sudo apt install postgresql`.

This will install postgres, which will start running on the background. We can acess it by running
`sudo -i -u postgres`, which will connect us. Note that there will be a change in the terminal, which
will indicate that you are now with the `postgres` user. From here, run `psql`, which
will start a postgres repl, where you can type SQL commands.

Let's create a database and a user.

Run `create database juliatutorial;`.

Followed by `create user julia with password 'julia123';`.


In [3]:
conn = LibPQ.Connection("dbname=juliatutorial user=julia password=julia123 host=localhost")

PostgreSQL connection (CONNECTION_OK) with parameters:
  user = julia
  password = ********************
  channel_binding = prefer
  dbname = juliatutorial
  host = localhost
  port = 5432
  client_encoding = UTF8
  options = -c DateStyle=ISO,YMD -c IntervalStyle=iso_8601 -c TimeZone=UTC
  application_name = LibPQ.jl
  sslmode = prefer
  sslcompression = 0
  sslsni = 1
  ssl_min_protocol_version = TLSv1.2
  gssencmode = prefer
  krbsrvname = postgres
  target_session_attrs = any

**Remember** to close the connection after finishing. 

In [4]:
close(conn)

## 2. The Basics

Let's now do some basic queries.


### 2.1 SQLStrings.jl
First, notice that we'll use the `SQLStrings.jl` package in order to parse our SQL queries. This is
necessary to avoid SQL injection attacks (read the docs on `SQLStrings.jl` to learn more).

In [5]:
function runquery(conn, sql::SQLStrings.Sql)
    query, args = SQLStrings.prepare(sql)
    LibPQ.execute(conn, query, args)
end

runquery (generic function with 1 method)

### 2.2 Creating a Table

Let's start by creating a new table and then deleting it. 

In [185]:
conn = LibPQ.Connection("dbname=juliatutorial user=julia password=julia123 host=localhost")
runquery(conn, sql`CREATE TABLE Employees (name char(64),age int,date_of_birth date,salary float)`)
runquery(conn, sql`drop TABLE Employees `)
close(conn)

### 2.3 Inserting Values in Table

Very easy. Let's create our table again and populate it. 

In [186]:
conn = LibPQ.Connection("dbname=juliatutorial user=julia password=julia123 host=localhost");
runquery(conn, sql`CREATE TABLE Employees (name char(64),age int,date_of_birth date,salary float)`)

PostgreSQL result

In [187]:
runquery(conn, sql`INSERT INTO Employees VALUES('James', 30, '1992-01-21', 3000.0)`)

PostgreSQL result

Note that the date format is `yyyy-mm-dd`.

Let's insert another row, but let's use string interpolation instead of writing the whole thing
by hand.

In [188]:
table = "Employees"
row   = Dict("name"=>"Bill", "date_of_birth"=>"2002-03-01", "age"=>20, "salary"=>2000)
row   = Dict("name"=>"Bill", "age"=>20, "date_of_birth"=>"2002-03-01", "salary"=>2000)

cols = ["name", "salary", "date_of_birth", "age"]
rowvalues = tuple([row[k] for k in cols]...)
runquery(conn,sql`INSERT INTO Employees (name, salary, date_of_birth, age) VALUES ($(rowvalues...))`)

PostgreSQL result

Let's now add a column with an id.

In [192]:
runquery(conn,sql`ALTER TABLE Employees ADD employee_id SERIAL PRIMARY KEY;`)

### 2.4 Selecting Table Values

Next, let's show how we can run queries to select the values inside a table.

In [193]:
runquery(conn, sql`SELECT * FROM Employees `) |> DataFrame

,name,age,date_of_birth,salary,employee_id
,String?,Int32?,Date,Float64?,Int32?
1,James,30,1992-01-21,3000.0,1
2,Bill,20,2002-03-01,2000.0,2


### 2.5 Showing Available Tables

How can we show the tables we have on our database "juliatutorial"? If we hop inside Postgres `psql` terminal,
we could just run the command `\dt`. This is not an option from within Julia. So we use a query
that excludes some auxiliary tables that are automatically generated by Postgres.

In [110]:
SHOWTABLES =  sql`
SELECT *
FROM pg_catalog.pg_tables
WHERE schemaname != 'pg_catalog' AND 
    schemaname != 'information_schema';
`

runquery(conn, SHOWTABLES) |> DataFrame

,schemaname,tablename,tableowner,tablespace,hasindexes,hasrules,hastriggers
,String?,String?,String?,String?,Bool?,Bool?,Bool?
1,public,employees,julia,missing,0,0,0


## 3. SQL and DataFrames
Let's now showcase how one can operate with SQL and Dataframes, doing things like populating an SQL table
from a DataFrame.

In [13]:
table = PalmerPenguins.load()
df = DataFrame(table)

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
,String15,String15,Float64?,Float64?,Int64?,Int64?
1,Adelie,Torgersen,39.1,18.7,181,3750
2,Adelie,Torgersen,39.5,17.4,186,3800
3,Adelie,Torgersen,40.3,18.0,195,3250
4,Adelie,Torgersen,missing,missing,missing,missing
5,Adelie,Torgersen,36.7,19.3,193,3450
6,Adelie,Torgersen,39.3,20.6,190,3650
7,Adelie,Torgersen,38.9,17.8,181,3625
8,Adelie,Torgersen,39.2,19.6,195,4675
9,Adelie,Torgersen,34.1,18.1,193,3475
